# Text Generation with Neural Networks

In this notebook we will create a network that can generate text and it is being done character by character. 

Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


+ [Step 1: The Data](#step1)
    + [Understanding unique characters](#vocab)
+ [Step 2: Text Processing](#step2)
    + [Text Vectorization](#vectorization)
        + [Character to Index](#chartoindex)
        + [Index to Character](#indextochar)
    + [Encoding the characters of the whole text](#encoding)
+ [Step 3: Creating Batches](#step3)
    + [First we need to understand the text we are using](#understand)
    + [Training Sequences](#trainingsequences)
        + [Create Training Sequences](#createtrainingsequences)
        + [Creating Sequence Batches](#createsequencebatches)
        + [Generating Training Batches](#generatingtrainingbatches)
+ [Step 4: Creating the Model](#step4)
    + [Setting up Loss Function](#setuplossfunction)
    + [Setting up Model Function](#modelfunction)
+ [Step 5: Training the modell](#step5)
+ [Step 6: Generating text](#step6)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

# <a name='step1'> Step 1: The Data </a>

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [14]:
path_to_file = '/content/drive/MyDrive/TS2/shakespeare.txt'

In [15]:
text = open(path_to_file, mode='r').read()

In [16]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

## <a name='vocab'>Understanding unique characters </a>

In [17]:
vocab = sorted(set(text))
vocab[:5]

['\n', ' ', '!', '"', '&']

In [18]:
len(vocab)

84

--------
# <a name='step2'>Step 2: Text Processing</a>

## <a name='vectorization'>Text Vectorization</a>

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

### <a name='chartoindex'>Character to Index</a>

In [19]:
char_to_index = {char:index for index,char in enumerate(vocab)}

In [20]:
char_to_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [21]:
char_to_index['H']

33

### <a name='indextochar'>Index to Character</a>

In [22]:
index_to_char = np.array(vocab)

In [23]:
index_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [24]:
index_to_char[11]

'0'

## <a name='encoding'>Encoding the characters of the whole text</a>
+ direct mapping for each character by character

In [25]:
encoded_text = np.array([char_to_index[c] for c in text])

In [26]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [27]:
encoded_text.shape

(5445609,)

We now have a mapping we can use to go back and forth from characters to numerics.

In [28]:
# Let's say for the sample text (Original Text)
sample = text[:500]
sample

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [29]:
# Let's see the encoded form of the above sample text, this is direct mapping character by character
encoded_sample = encoded_text[:500]
encoded_sample

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64, 62, 63,
       75,  1, 69, 60, 77, 60, 73,  1, 59, 64, 60,  8,  0,  1,  1, 27, 76,
       75,  1, 56, 74,  1, 75, 63, 60,  1, 73, 64, 71, 60, 73,  1, 74, 63,
       70, 76, 67, 59,  1, 57, 80,  1, 75, 64, 68, 60,  1, 59, 60, 58, 60,
       56, 74, 60,  8,  0,  1,  1, 33, 64, 74,  1, 75, 60, 69, 59, 60, 73,
        1, 63, 60, 64, 73,  1, 68, 64, 62, 63, 75,  1, 57, 60, 56, 73,  1,
       63, 64, 74,  1, 68, 60, 68, 70, 73, 80, 21,  0,  1,  1, 27, 76, 75,
        1, 75, 63, 70, 76,  1, 58, 70, 69, 75, 73, 56, 58, 75, 60, 59,  1,
       75, 70,  1, 75, 63

---------

# <a name='step3'>Step 3: Creating Batches</a>

+ Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. 
+ Its up to us (the user) to choose how long that historic sequence. 
    + Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , 
    + too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. 
+ While there is no correct sequence length choice, you should consider 
    + the text itself, 
    + how long normal phrases are in it, and 
    + a reasonable idea of what characters/words are relevant to each other.

## <a name='understand'>First we need to understand the text we are using</a>

In [30]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [31]:
lines = 'From fairest creatures we desire increase,'
len(lines)

42

As Shakespears writing's fortunatley include rhymes between the lines, we need to pick up at least 3 lines to pick up those patterns.

Example: check out the ending words. They do rhyme, such as `increase, decrease` , `die , eyes, lies`

There are about 134 characters for 3 lines, one line is about 42 characters.

In [32]:
lines = '''
 From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
'''

len(lines)

134

------

## <a name='trainingsequences'>Training Sequences</a>

The actual text data will be the text sequence shifted one character forward. For example:

**Sequence In: "Hello my nam"
Sequence Out: "ello my name"**


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

**We will use about 120 characters for the sequence length.**

In [33]:
sequence_length = 120

In [34]:
# calculate how many sequences for the whole text, becuse of 0 indexing we will add 1
total_num_sequence = len(text) // (sequence_length+1)

In [35]:
total_num_sequence

45005

### <a name='createtrainingsequences'>Create Training Sequences</a>

In [36]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [37]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [38]:
# grabbing data from training sequences - demo for first 500 characters
for item in char_dataset.take(500):
    print(index_to_char[item.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


### <a name='createsequencebatches'>Creating Sequence Batches</a>

The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use `seq_len+1` because of zero indexing. Here is what `drop_remainder` means:

+ drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


Why we have remainder in the first place?
+ when we divide length of the whole text by 120 (sequence length), there are some remainder left. This is because the length of whole text is not perfectly divisible by 120, which is expected. We will choose to discard those remaining chacters which are not part of any sequence group. This won't have impact on our training as our text is already quite large.

In [39]:
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

In [40]:
sequences

<BatchDataset shapes: (121,), types: tf.int64>

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [41]:
def create_sequence_targets(seq):
    input_txt = seq[:-1] # Hello my nam
    target_txt = seq[1:] # ello my name
    return (input_txt, target_txt)

In [42]:
dataset = sequences.map(tf.autograph.experimental.do_not_convert(create_sequence_targets))

#### Demo of how it looks like for one sequence of first batch
+ we can see input_txt is starting with 0 and end with 75.
+ for target_txt is shfiting to next 1 character, starting with 1 and ending with 1

In [43]:
for input_txt, target_txt in dataset.take(1): # get the one sequence batch
    # for input text
    print(input_txt.numpy())
    print("".join(index_to_char[input_txt.numpy()]))
    
    print('\n')
    
    # for target text
    print(target_txt.numpy())
    print("".join(index_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### <a name='generatingtrainingbatches'>Generating Training Batches</a>

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [44]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle the entire sequence in memory. 
# Instead, it maintains a buffer in which it shuffles elements
buffer_size = 1000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)


In [45]:
dataset

# we can see input sequence and target sequence

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

-------

# <a name='step4'>Step 4: Creating the Model</a>

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "`embedding dim`" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [52]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension (We might want to choose same scale of vocab size)
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

In [53]:
vocab_size 

84

## <a name='setuplossfunction'> Setting up Loss Function</a>

For our loss we will use sparse categorical crossentropy, which we can import from Keras. **We will also set this as `logits=True` as our data are `one-hot encoded`**.

In [54]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [55]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

## <a name='modelfunction'> Setting up Model Function</a>

Now let's create a function that easily adapts to different variables as shown above.

In [56]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, GRU

In [57]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    
    model.add(GRU(rnn_neurons,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'))
    
    model.add(Dense(vocab_size))
    
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    
    return model

In [58]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size)

In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


----

# <a name='step5'>Step 5: Training the model</a>

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.


In [63]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [64]:
example_batch_predictions.shape

TensorShape([128, 120, 84])

In [65]:
example_batch_predictions[0]

<tf.Tensor: shape=(120, 84), dtype=float32, numpy=
array([[ 0.00744317, -0.00090096, -0.0094114 , ...,  0.00486787,
        -0.00405866, -0.00121064],
       [ 0.00855853, -0.00498477, -0.00884936, ...,  0.00592785,
        -0.00123757, -0.00812292],
       [ 0.00321869, -0.00558345, -0.00278171, ...,  0.00023974,
        -0.00402449,  0.00210256],
       ...,
       [ 0.0038353 ,  0.00550529,  0.00038914, ..., -0.00625554,
        -0.00011505, -0.01148627],
       [ 0.00331539, -0.00225345,  0.0048323 , ...,  0.00185895,
        -0.0020827 , -0.01181064],
       [ 0.00576036, -0.00651682, -0.00320568, ...,  0.00465379,
         0.00060987, -0.01383383]], dtype=float32)>

These are just probabilities for each characters.


In [66]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [67]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[34],
       [44],
       [39],
       [39],
       [68],
       [ 9],
       [38],
       [57],
       [25],
       [49],
       [ 5],
       [20],
       [57],
       [69],
       [65],
       [18],
       [27],
       [39],
       [73],
       [ 4],
       [12],
       [ 9],
       [63],
       [20],
       [21],
       [20],
       [ 8],
       [20],
       [77],
       [55],
       [ 1],
       [44],
       [45],
       [11],
       [32],
       [73],
       [61],
       [58],
       [42],
       [59],
       [83],
       [39],
       [79],
       [49],
       [ 0],
       [66],
       [21],
       [28],
       [51],
       [45],
       [39],
       [77],
       [71],
       [71],
       [57],
       [22],
       [72],
       [51],
       [ 9],
       [16],
       [53],
       [59],
       [41],
       [69],
       [72],
       [78],
       [17],
       [53],
       [ 5],
       [41],
       [58],
       [57],
       [32],
   

In [68]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [69]:
sampled_indices

array([34, 44, 39, 39, 68,  9, 38, 57, 25, 49,  5, 20, 57, 69, 65, 18, 27,
       39, 73,  4, 12,  9, 63, 20, 21, 20,  8, 20, 77, 55,  1, 44, 45, 11,
       32, 73, 61, 58, 42, 59, 83, 39, 79, 49,  0, 66, 21, 28, 51, 45, 39,
       77, 71, 71, 57, 22, 72, 51,  9, 16, 53, 59, 41, 69, 72, 78, 17, 53,
        5, 41, 58, 57, 32, 57, 15, 57, 83,  3, 43, 81, 47, 65, 66, 16, 74,
        1, 72, 51, 52, 61, 22, 37, 52, 11, 29,  3, 83,  3, 24, 18, 42, 62,
        6, 28, 16, 75, 53, 82, 27, 71, 43,  8, 37,  4, 23, 29, 27, 34, 72,
       78])

In [71]:
print("Given the input seq: \n")
print("".join(index_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(index_to_char[sampled_indices ]))


Given the input seq: 

d him from me now.
    Yet him for this, my love no whit disdaineth,
    Suns of the world may stain, when heaven's sun 


Next Char Predictions: 

ISNNm-Mb?X'9bnj7BNr&1-h9:9,9v` ST0GrfcQd}NxX
k:CZTNvppb;qZ-5]dPnqw6]'PcbGb4b}"RzVjk5s qZ[f;L[0D"}">7Qg(C5t]|BpR,L&<DBIqw


We can see these are just random predictions of characters as we haven't trained our model yet.

After confirming the dimensions are working, let's train our network!

In [72]:
epochs = 30

In [73]:
model.fit(dataset, epochs = epochs)

Epoch 1/30
351/351 [==============================] - 43s 119ms/step - loss: 3.1474
Epoch 2/30
351/351 [==============================] - 43s 122ms/step - loss: 1.8723
Epoch 3/30
351/351 [==============================] - 43s 124ms/step - loss: 1.5653
Epoch 4/30
351/351 [==============================] - 45s 128ms/step - loss: 1.4094
Epoch 5/30
351/351 [==============================] - 45s 129ms/step - loss: 1.3279
Epoch 6/30
351/351 [==============================] - 46s 132ms/step - loss: 1.2776
Epoch 7/30
351/351 [==============================] - 45s 129ms/step - loss: 1.2428
Epoch 8/30
351/351 [==============================] - 47s 132ms/step - loss: 1.2154
Epoch 9/30
351/351 [==============================] - 45s 129ms/step - loss: 1.1911
Epoch 10/30
351/351 [==============================] - 46s 129ms/step - loss: 1.1706
Epoch 11/30
351/351 [==============================] - 45s 129ms/step - loss: 1.1527
Epoch 12/30
351/351 [==============================] - 45s 128ms/step - lo

--------

# <a name='step6'>Step 6: Generating text</a>

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [75]:
model.save('shakespeare_gen.h5')

In [76]:
from tensorflow.keras.models import load_model

In [77]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [78]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [83]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_index[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(index_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [90]:
print(generate_text(model,"love",gen_size=1000))

loves; and anon-
    She had no sintine, Greek,
    Whose witchcraster hath bounds me blind,
  With this, this letter, that they ride so well?
  CLOWN. I met a fool and to say so; the first time when she died the
    you would tarry under the Prince; that there was to be found, and
    both the maid for tailing any left hand,
    We'll encompass out of you to my request,
  Must be against kind and one that keeps the Tears of my forsune.
    Love cannot have you free and hold it
    The enemies to me. If we be his
    sister- how and ye whose searcher hast thou forg'd i' th' care.
  TAMORA. My husar, half heard me; all, true one,
    Were nothing benefits   Thou are those credies upon you? You have done
    How hus more fresh! my mother, what thou art?     ospers.

  Con, I have avoided
    As advesture seems down and safe to thee;
    So IOr I'll corrupt the dead in this. All's with her;
    And I envy arives you crowned in holy execution;
    two and twenty shogors and the reing A per